In [ ]:
'd123ea55-ce1d-4fe0-b887-f58fef5c14a9'

In [2]:
import requests
import pandas as pd
from io import StringIO

API_KEY = 'd123ea55-ce1d-4fe0-b887-f58fef5c14a9'  # <-- put your token here

# Example: Day-ahead prices for DE-LU (bidding zone EIC)
# Time window is UTC, format YYYYMMDDHHMM, end is exclusive.
params = {
    "securityToken": API_KEY,
    "documentType": "A44",                # Day-ahead prices
    "in_Domain":  "10Y1001A1001A63L",     # DE-LU EIC
    "out_Domain": "10Y1001A1001A63L",     # same as in_Domain
    "periodStart": "202511130000",        # UTC
    "periodEnd":   "202511140000",        # UTC (exclusive)
}
url = "https://web-api.tp.entsoe.eu/api"

resp = requests.get(url, params=params, timeout=60)
resp.raise_for_status()  # raise if the API returned an error

# Parse XML to DataFrame
xml = resp.text

# 1) Extract the hourly "Point" rows (position, price.amount)
points = pd.read_xml(StringIO(xml), xpath=".//TimeSeries/Period/Point")
# 2) Extract the Period's start and resolution (assume PT60M for A44)
periods = pd.read_xml(StringIO(xml), xpath=".//TimeSeries/Period")

# If multiple Period blocks exist, repeat each start for its number of points
if not periods.empty and not points.empty:
    # Forward-fill period start times to each point (simple approach)
    # Each Period contains a list of Points in order; we align by cumulative lengths.
    periods["n_points"] = periods["Point"].apply(lambda x: len(x) if isinstance(x, list) else 0) if "Point" in periods else 0
    # Fallback: if "Point" column not present (depends on pandas version), assume hourly count from response
    if "n_points" not in periods or periods["n_points"].sum() == 0:
        # Best-effort: assume 24 hourly points for day-ahead
        periods["n_points"] = 24

    period_starts = []
    for _, row in periods.iterrows():
        period_starts.extend([row["timeInterval.start"]]*int(row["n_points"]))
    # Trim/extend to match the number of points
    period_starts = period_starts[:len(points)] if len(period_starts) >= len(points) else period_starts + [period_starts[-1]]*(len(points)-len(period_starts))
    points["period_start"] = pd.to_datetime(period_starts, utc=True)

# Build timestamps from position (1..N) and hourly resolution
points["position"] = points["position"].astype(int)
points["time_utc"] = points["period_start"] + pd.to_timedelta(points["position"] - 1, unit="h")

# Final tidy frame
df = points.rename(columns={"price.amount": "price_eur_mwh"})[["time_utc", "price_eur_mwh"]].sort_values("time_utc").reset_index(drop=True)
print(df.head())


SSLError: HTTPSConnectionPool(host='web-api.tp.entsoe.eu', port=443): Max retries exceeded with url: /api?securityToken=d123ea55-ce1d-4fe0-b887-f58fef5c14a9&documentType=A44&in_Domain=10Y1001A1001A63L&out_Domain=10Y1001A1001A63L&periodStart=202511130000&periodEnd=202511140000 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1010)')))